In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import statsmodels as sm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.linear_model import LogisticRegression
from statsmodels.discrete.discrete_model import Logit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score, f1_score, classification_report, confusion_matrix
from statsmodels.tools.tools import add_constant
from itertools import product, chain, combinations

warnings.simplefilter(action='ignore')

In [2]:
def profit_score (conf_matrix, d=0.2, lgd=0.8):
    tn, fp, fn = conf_matrix[0][0], conf_matrix[0][1], conf_matrix[1][0]
    return (tn*d - fn*lgd)/(d*(fp+tn))

def FNR(conf_matrix):
    tn, fn = conf_matrix[0][0], conf_matrix[1][0]
    return (fn/(tn+fn))

def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [9]:
def gen_dataset(data, target, main_feats, feats_add, feats_encod, regr=True, return_feats=False):
    temp_data = data.copy()
    if feats_encod != 'False':
        origin_feats = list(temp_data.columns)
        temp_data = pd.get_dummies(temp_data, columns=feats_encod, drop_first=True)
        new_feats = [c for c in temp_data.columns if c not in origin_feats]
        temp_data = temp_data[main_feats+feats_add+new_feats]
    else:
        temp_data = temp_data[main_feats+feats_add]
        
    X_train, X_test, y_train, y_test = train_test_split(temp_data, target, test_size=0.15, random_state=42)
    
    if regr:
        sc = StandardScaler()
        X_train[main_feats+feats_add] = sc.fit_transform(X_train[main_feats+feats_add])
        X_test[main_feats+feats_add] = sc.transform(X_test[main_feats+feats_add])        
    if return_feats:
        return X_train, X_test, y_train, y_test, list(X_train.columns)
    else:
        return X_train, X_test, y_train, y_test

In [64]:
feats_list = [['mob', 'MOB_term', 'Credit_TermApr', 'curr_del_cap_share_one', 'maxdelay_one',
        'MA_AGE', 'MA_MONTH_AT_CURR_ADDRESS', 'MA_MONTH_AT_CURR_PASSP', 
        'MA_MONTH_AT_CURR_JOB', 'MA_Time_Previous_Job',
       'MA_Installment_Amount', 'MA_Proposed_Amount',
       'TOT_INCOME','delays_one'],
              
              ['mob', 'MOB_term', 'Credit_TermApr', 'curr_del_cap_share_one', 'maxdelay_one',
        'MA_AGE', 'MA_MONTH_AT_CURR_ADDRESS', 'MA_MONTH_AT_CURR_PASSP', 
        'MA_MONTH_AT_CURR_JOB', 'MA_Time_Previous_Job','MA_Installment_Amount','delays_one', 'age_range'],
              
              ['mob', 'Credit_TermApr', 'curr_del_cap_share_one', 'maxdelay_one',
        'MA_AGE', 'MA_MONTH_AT_CURR_ADDRESS', 'MA_MONTH_AT_CURR_PASSP', 
        'MA_MONTH_AT_CURR_JOB', 'MA_Time_Previous_Job',
       'MA_Installment_Amount','TOT_INCOME', 'delays_one'],
              
              ['mob','Credit_TermApr', 'maxdelay_one',
        'MA_AGE', 'MA_MONTH_AT_CURR_ADDRESS', 'MA_MONTH_AT_CURR_PASSP', 'MA_MONTH_AT_CURR_JOB', 'MA_Time_Previous_Job',
       'MA_Installment_Amount','TOT_INCOME','curr_profit_per', 'ratio_curr_cap_share', 'delays_one'],
              
              ['mob', 'MOB_term', 'curr_rep_cap_share_one', 'maxdelay_one',
        'MA_AGE', 'MA_MONTH_AT_CURR_ADDRESS', 'MA_MONTH_AT_CURR_PASSP','MA_MONTH_AT_CURR_JOB', 'MA_Time_Previous_Job',
       'MA_Installment_Amount','TOT_INCOME','all_profit_per', 'delays_one'],
              
                ['mob',  'curr_rep_cap_share_one', 'maxdelay_one',
        'MA_AGE', 'MA_MONTH_AT_CURR_ADDRESS', 'MA_MONTH_AT_CURR_PASSP',
               'MA_MONTH_AT_CURR_JOB', 'MA_Time_Previous_Job', 'MA_Proposed_Amount',
       'TOT_INCOME', 'all_profit_per', 'delays_one'],
              
                ['mob',  'MOB_term', 'maxdelay_one',
        'MA_AGE', 'MA_MONTH_AT_CURR_ADDRESS', 'MA_MONTH_AT_CURR_PASSP',
               'MA_MONTH_AT_CURR_JOB', 'MA_Time_Previous_Job',
       'MA_Installment_Amount',
       'TOT_INCOME', 'all_profit_per', 'ratio_curr_cap_share', 'delays_one']]

feats_to_search = ['PARTWH_INCOME', 'PAYMD2TOTPAYM', 'PTI', 'prof_per_month', 'ratio_inst_amount', 'diff_white_pti', 'ratio_time_job', 'ratio_amount_income','ration_del_cr']

feats_to_ohe_list = ['False', 
                    ['MA_Gender', 'MA_Education','MA_Marital_Status','MANUMBEROFCHILD', 'MA_Residential_Status','MA_Real_Estate_Owner','MA_REG_Same_Fact_Addr', 'MA_Exp_IND'],
                    ['MA_Gender', 'MA_Education', 'MA_Residential_Status','MA_REG_Same_Fact_Addr'],
                    ['MA_Education','MA_Marital_Status','MA_Exp_IND'],
                    ['MA_Education','MA_Marital_Status', 'MA_Residential_Status']]

In [3]:
data = pd.read_csv('data/clean_data_feats.csv')
target = data['bad']
data.drop(['bad'], axis=1, inplace=True)

In [92]:
def best_regr(data, target, feats_list, feats_to_search, feats_ohe_list):
    metrics_dict = {'roc': [0, 0],
                    'w_f1': [0, 0],
                    'profit': [0, 0],
                    'FNR': [0, 0]
                   }
    
    max_roc, max_wf1, max_prof, min_fnr = 0, 0, 0, 1
    for main_feats in feats_list:    
        for feat in map(list, powerset(feats_to_search)):
            for feats_ohe in feats_ohe_list:
                temp_data = data.copy()
                
        
                if feats_ohe != 'False':
                    origin_feats = list(temp_data.columns)
                    temp_data = pd.get_dummies(temp_data, columns=feats_ohe, drop_first=True)
                    new_feats = [c for c in temp_data.columns if c not in origin_feats]
                    temp_data = temp_data[main_feats+feat+new_feats]
                else:
                    temp_data = temp_data[main_feats+feat]
                
                X_train, X_test, y_train, y_test = train_test_split(temp_data, target, test_size=0.15, random_state=42)

                sc = StandardScaler()
                X_train[main_feats+feat] = sc.fit_transform(X_train[main_feats+feat])
                X_test[main_feats+feat] = sc.transform(X_test[main_feats+feat])

                m = Logit(y_train, X_train).fit(disp=0)
                pred = m.predict(X_test)

                roc = roc_auc_score(y_test, pred)
                labels =  np.array(pred > 0.5, dtype=float)
                wf1 = f1_score(y_test, labels, average='weighted')
                prof = profit_score(confusion_matrix(y_test, labels))
                fnr = FNR(confusion_matrix(y_test, labels))


                if max_roc<roc:
                    max_roc = roc
                    metrics_dict['roc'][0] = max_roc
                    metrics_dict['roc'][1] = [main_feats, feat, feats_ohe]

                if max_wf1<wf1:
                    max_wf1 = wf1
                    metrics_dict['w_f1'][0] = max_wf1
                    metrics_dict['w_f1'][1] = [main_feats, feat, feats_ohe]

                if max_prof<prof:
                    max_prof = prof
                    metrics_dict['profit'][0] = max_prof
                    metrics_dict['profit'][1] = [main_feats, feat, feats_ohe]

                if min_fnr>fnr:
                    min_fnr = fnr
                    metrics_dict['FNR'][0] = min_fnr
                    metrics_dict['FNR'][1] = [main_feats, feat, feats_ohe]
                
    return metrics_dict

In [93]:
%%time
md = best_regr(data, target, feats_list, feats_to_search, feats_to_ohe_list)

Wall time: 58min


In [94]:
md

{'roc': [0.7435224518331612,
  [['mob',
    'Credit_TermApr',
    'maxdelay_one',
    'MA_AGE',
    'MA_MONTH_AT_CURR_ADDRESS',
    'MA_MONTH_AT_CURR_PASSP',
    'MA_MONTH_AT_CURR_JOB',
    'MA_Time_Previous_Job',
    'MA_Installment_Amount',
    'TOT_INCOME',
    'curr_profit_per',
    'ratio_curr_cap_share',
    'delays_one'],
   ['PAYMD2TOTPAYM', 'ratio_amount_income'],
   'False']],
 'w_f1': [0.7560637242148956,
  [['mob',
    'MOB_term',
    'curr_rep_cap_share_one',
    'maxdelay_one',
    'MA_AGE',
    'MA_MONTH_AT_CURR_ADDRESS',
    'MA_MONTH_AT_CURR_PASSP',
    'MA_MONTH_AT_CURR_JOB',
    'MA_Time_Previous_Job',
    'MA_Installment_Amount',
    'TOT_INCOME',
    'all_profit_per',
    'delays_one'],
   ['PAYMD2TOTPAYM',
    'PTI',
    'prof_per_month',
    'diff_white_pti',
    'ratio_time_job',
    'ratio_amount_income'],
   ['MA_Gender',
    'MA_Education',
    'MA_Residential_Status',
    'MA_REG_Same_Fact_Addr']]],
 'profit': [0.39005113900511396,
  [['mob',
    'Credit_Ter

In [87]:
def best_regr_skl(data, target, feats_list, feats_to_search, feats_ohe_list):
    metrics_dict = {'roc': [0, 0],
                    'w_f1': [0, 0],
                    'profit': [0, 0],
                    'FNR': [0, 0]
                   }
    
    max_roc, max_wf1, max_prof, min_fnr = 0, 0, 0, 1
    for main_feats in feats_list:    
        for feat in map(list, powerset(feats_to_search)):
            for feats_ohe in feats_ohe_list:
                temp_data = data.copy()
                
        
                if feats_ohe != 'False':
                    origin_feats = list(temp_data.columns)
                    temp_data = pd.get_dummies(temp_data, columns=feats_ohe, drop_first=True)
                    new_feats = [c for c in temp_data.columns if c not in origin_feats]
                    temp_data = temp_data[main_feats+feat+new_feats]
                else:
                    temp_data = temp_data[main_feats+feat]
                
                X_train, X_test, y_train, y_test = train_test_split(temp_data, target, test_size=0.15, random_state=42)

                sc = StandardScaler()
                X_train[main_feats+feat] = sc.fit_transform(X_train[main_feats+feat])
                X_test[main_feats+feat] = sc.transform(X_test[main_feats+feat])

                m = LogisticRegression(penalty='l2', solver='lbfgs', random_state=0,
                                     class_weight='balanced').fit(X_train, y_train)
                
                pred = m.predict(X_test)

                roc = roc_auc_score(y_test, pred)
                labels =  np.array(pred > 0.5, dtype=float)
                wf1 = f1_score(y_test, labels, average='weighted')
                prof = profit_score(confusion_matrix(y_test, labels))
                fnr = FNR(confusion_matrix(y_test, labels))


                if max_roc<roc:
                    max_roc = roc
                    metrics_dict['roc'][0] = max_roc
                    metrics_dict['roc'][1] = [main_feats, feat, feats_ohe]

                if max_wf1<wf1:
                    max_wf1 = wf1
                    metrics_dict['w_f1'][0] = max_wf1
                    metrics_dict['w_f1'][1] = [main_feats, feat, feats_ohe]

                if max_prof<prof:
                    max_prof = prof
                    metrics_dict['profit'][0] = max_prof
                    metrics_dict['profit'][1] = [main_feats, feat, feats_ohe]

                if min_fnr>fnr:
                    min_fnr = fnr
                    metrics_dict['FNR'][0] = min_fnr
                    metrics_dict['FNR'][1] = [main_feats, feat, feats_ohe]
                
    return metrics_dict

In [88]:
%%time
md_skl = best_regr_skl(data, target, feats_list, feats_to_search, feats_to_ohe_list)

Wall time: 48min 21s


In [89]:
md_skl

{'roc': [0.6899866385597325,
  [['mob',
    'MOB_term',
    'curr_rep_cap_share_one',
    'maxdelay_one',
    'MA_AGE',
    'MA_MONTH_AT_CURR_ADDRESS',
    'MA_MONTH_AT_CURR_PASSP',
    'MA_MONTH_AT_CURR_JOB',
    'MA_Time_Previous_Job',
    'MA_Installment_Amount',
    'TOT_INCOME',
    'all_profit_per',
    'delays_one'],
   ['PAYMD2TOTPAYM', 'prof_per_month', 'ratio_amount_income'],
   'False']],
 'w_f1': [0.7558315161106234,
  [['mob',
    'MOB_term',
    'maxdelay_one',
    'MA_AGE',
    'MA_MONTH_AT_CURR_ADDRESS',
    'MA_MONTH_AT_CURR_PASSP',
    'MA_MONTH_AT_CURR_JOB',
    'MA_Time_Previous_Job',
    'MA_Installment_Amount',
    'TOT_INCOME',
    'all_profit_per',
    'ratio_curr_cap_share',
    'delays_one'],
   ['prof_per_month',
    'ratio_inst_amount',
    'ratio_time_job',
    'ratio_amount_income'],
   'False']],
 'profit': [0.38958623895862393,
  [['mob',
    'MOB_term',
    'curr_rep_cap_share_one',
    'maxdelay_one',
    'MA_AGE',
    'MA_MONTH_AT_CURR_ADDRESS',
    '

In [116]:
df_sm = pd.DataFrame(columns=['roc', 'w_f1', 'FNR', 'Profit'])
for metric, feats in md.items():
    X_train, X_test, y_train, y_test = gen_dataset(data, target, feats[1][0], feats[1][1], feats[1][2])
    m = Logit(y_train, X_train).fit(disp=0)
    pred = m.predict(X_test)

    roc = roc_auc_score(y_test, pred)
    labels =  np.array(pred > 0.5, dtype=float)
    wf1 = f1_score(y_test, labels, average='weighted')
    prof = profit_score(confusion_matrix(y_test, labels))
    fnr = FNR(confusion_matrix(y_test, labels))
    tdf = pd.DataFrame({'roc': roc, 'w_f1': wf1, 'FNR': fnr, 'Profit': prof,},index=['best_by_'+metric])
    df_sm = pd.concat([df_sm, tdf])
    
df_sm

,roc,w_f1,FNR,Profit
best_by_roc,0.743522,0.732170,0.104408,0.383078
best_by_w_f1,0.698699,0.756064,0.178245,0.127383
best_by_profit,0.737726,0.739450,0.104333,0.390051
best_by_FNR,0.739404,0.729950,0.102579,0.386332


In [117]:
df_skl = pd.DataFrame(columns=['roc', 'w_f1', 'FNR', 'Profit'])
for metric, feats in md_skl.items():
    X_train, X_test, y_train, y_test = gen_dataset(data, target, feats[1][0], feats[1][1], feats[1][2])
    m = LogisticRegression(penalty='l2', solver='lbfgs', random_state=0,
                                     class_weight='balanced').fit(X_train, y_train)
    pred = m.predict(X_test)

    roc = roc_auc_score(y_test, pred)
    labels =  np.array(pred > 0.5, dtype=float)
    wf1 = f1_score(y_test, labels, average='weighted')
    prof = profit_score(confusion_matrix(y_test, labels))
    fnr = FNR(confusion_matrix(y_test, labels))
    tdf = pd.DataFrame({'roc': roc, 'w_f1': wf1, 'FNR': fnr, 'Profit': prof},index=['by_'+metric])
    df_skl = pd.concat([df_skl, tdf])
    
df_skl

,roc,w_f1,FNR,Profit
by_roc,0.689987,0.751124,0.108064,0.389586
by_w_f1,0.686953,0.755832,0.111231,0.384007
by_profit,0.689987,0.751124,0.108064,0.389586
by_FNR,0.689987,0.751124,0.108064,0.389586


In [8]:
best_feats_sm = [['mob',
    'Credit_TermApr',
    'maxdelay_one',
    'MA_AGE',
    'MA_MONTH_AT_CURR_ADDRESS',
    'MA_MONTH_AT_CURR_PASSP',
    'MA_MONTH_AT_CURR_JOB',
    'MA_Time_Previous_Job',
    'MA_Installment_Amount',
    'TOT_INCOME',
    'curr_profit_per',
    'ratio_curr_cap_share',
    'delays_one'],
   ['ratio_inst_amount',
    'diff_white_pti',
    'ratio_time_job',
    'ratio_amount_income'],
   'False']

best_feats_sk = [['mob',
    'MOB_term',
    'curr_rep_cap_share_one',
    'maxdelay_one',
    'MA_AGE',
    'MA_MONTH_AT_CURR_ADDRESS',
    'MA_MONTH_AT_CURR_PASSP',
    'MA_MONTH_AT_CURR_JOB',
    'MA_Time_Previous_Job',
    'MA_Installment_Amount',
    'TOT_INCOME',
    'all_profit_per',
    'delays_one'],
   ['PAYMD2TOTPAYM', 'prof_per_month', 'ratio_amount_income'],
   'False']

In [125]:
%%time
# Hypertuning of logit
X_train, X_test, y_train, y_test = gen_dataset(data, target, md_skl['profit'][1][0], md_skl['profit'][1][1], md_skl['profit'][1][2])
c_list = np.linspace(0.4, 0.6, num=100)
tol_list = [0.01, 0.1]

max_par = [0, 0, 0, 0]
for c in c_list:
    for tol in tol_list:
        log_reg = LogisticRegression(penalty='l2', solver='lbfgs', C=c, tol=tol, random_state=0,
                                     class_weight='balanced').fit(X_train, y_train)
        pred = log_reg.predict(X_test)
        labels =  np.array(pred > 0.5, dtype=float)
        score = profit_score(confusion_matrix(y_test, labels))
        roc = roc_auc_score(y_test, pred)
        if max_par[2] < score:
            max_par[0] = c
            max_par[1] = tol
            max_par[2] = score
            max_par[3] = roc
        #print('C: {}, tol: {}, score: {:<8.7f}'.format(c, tol, score, roc))
print('C: {}, tol: {}, score: {:<8.7f}, {:<8.7f}'.format(max_par[0], max_par[1], max_par[2], max_par[3]))

C: 0.5555555555555556, tol: 0.01, score: 0.3895862, 0.6899866
Wall time: 7.62 s


### statsmodel logit

In [132]:
X_train, X_test, y_train, y_test = gen_dataset(data, target, best_feats_sm[0], best_feats_sm[1], best_feats_sm[2])
regr_sm = Logit(y_train, X_train).fit()
print(regr_sm.pred_table())
regr_sm.summary()

Optimization terminated successfully.
         Current function value: 0.651231
         Iterations 7
[[8957. 3258.]
 [1075. 1865.]]


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    bad   No. Observations:                15155
Model:                          Logit   Df Residuals:                    15138
Method:                           MLE   Df Model:                           16
Date:                Sun, 22 May 2022   Pseudo R-squ.:                 -0.3237
Time:                        22:30:18   Log-Likelihood:                -9869.4
converged:                       True   LL-Null:                       -7455.7
Covariance Type:            nonrobust   LLR p-value:                     1.000
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
mob                         -0.0294      0.055     -0.539      0.590      -0.136       0.078
Credit_TermApr              -0.0243      0.035     -0.702      0.482      -0.092       0.043
maxdelay_one                 0.9478      0.037     25.377      0.000       0.875       1.021
MA_AGE                       0.1035      0.020      5.093      0.000       0.064       0.143
MA_MONTH_AT_CURR_ADDRESS    -0.0064      0.018     -0.358      0.721      -0.042       0.029
MA_MONTH_AT_CURR_PASSP      -0.0439      0.017     -2.570      0.010      -0.077      -0.010
MA_MONTH_AT_CURR_JOB        -0.0399      0.019     -2.059      0.039      -0.078      -0.002
MA_Time_Previous_Job         0.0029      0.022      0.130      0.896      -0.040       0.046
MA_Installment_Amount        0.1387      0.037      3.704      0.000       0.065       0.212
TOT_INCOME                  -0.0861      0.031     -2.796      0.005      -0.147      -0.026
curr_profit_per             -0.3212      0.076     -4.242      0.000      -0.470      -0.173
ratio_curr_cap_share         0.4733      0.087      5.466      0.000       0.304       0.643
delays_one                  -0.1763      0.028     -6.303      0.000      -0.231      -0.121
ratio_inst_amount           -0.0304      0.032     -0.959      0.338      -0.093       0.032
diff_white_pti              -0.0145      0.019     -0.769      0.442      -0.052       0.022
ratio_time_job               0.0010      0.022      0.046      0.963      -0.042       0.044
ratio_amount_income          0.0186      0.031      0.604      0.546      -0.042       0.079
============================================================================================
"""

In [133]:
proba_sm = regr_sm.predict(X_test)
print('roc:  ',roc_auc_score(y_test, proba_sm))
pred_sm = np.array(proba_sm > 0.5, dtype=float) 
print(classification_report(y_test, pred_sm))

roc:   0.7377262838871321
              precision    recall  f1-score   support

           0       0.90      0.73      0.80      2151
           1       0.37      0.65      0.47       524

    accuracy                           0.71      2675
   macro avg       0.63      0.69      0.64      2675
weighted avg       0.79      0.71      0.74      2675



In [134]:
print('Profit: ', profit_score(confusion_matrix(y_test, pred_sm)))
print('FNR: ', FNR(confusion_matrix(y_test, pred_sm)))
print('wf1: ', f1_score(y_test, pred_sm, average='weighted'))

Profit:  0.39005113900511396
FNR:  0.10433295324971494
wf1:  0.7394498974218852


### sklearn logit

In [138]:
X_train, X_test, y_train, y_test, name_feats = gen_dataset(data, target, best_feats_sk[0], best_feats_sk[1], best_feats_sk[2], return_feats=True)
regr_sk = LogisticRegression(penalty='l2', solver='lbfgs', C=c, tol=tol, random_state=0,
                                     class_weight='balanced').fit(X_train, y_train)
proba_sk = regr_sk.predict(X_test)

print('roc:  ',roc_auc_score(y_test, proba_sk))

pred_sk = np.array(proba_sk > 0.5, dtype=float) 
print(classification_report(y_test, pred_sk))

roc:   0.6899866385597325
              precision    recall  f1-score   support

           0       0.89      0.76      0.82      2151
           1       0.38      0.62      0.48       524

    accuracy                           0.73      2675
   macro avg       0.64      0.69      0.65      2675
weighted avg       0.79      0.73      0.75      2675



In [131]:
print('Profit: ', profit_score(confusion_matrix(y_test, pred_sk)))
print('FNR: ', FNR(confusion_matrix(y_test, pred_sk)))
print('wf1: ', f1_score(y_test, pred_sk, average='weighted'))

Profit:  0.38958623895862393
FNR:  0.10806363137685135
wf1:  0.7511239826969192


In [146]:
pd.DataFrame({'Feature': name_feats, 'Coef': regr_sk.coef_[0]}).sort_values(by="Coef",key=abs,ascending=False)

,Feature,Coef
3,maxdelay_one,1.168213
1,MOB_term,0.720848
2,curr_rep_cap_share_one,-0.514127
0,mob,-0.356277
12,delays_one,-0.220951
13,PAYMD2TOTPAYM,-0.193649
4,MA_AGE,0.154973
10,TOT_INCOME,-0.138890
11,all_profit_per,0.134264
9,MA_Installment_Amount,0.123515


### const model

In [11]:
# пример плохого алгоритма (все 0)
conf_matr = np.array([[2156, 0], [519, 0]])
labels = np.zeros(2675) #2675 число элементов в тесте
print(conf_matr)
print('roc: ', roc_auc_score(y_test, labels))
print('wf1: ', f1_score(y_test, labels, average='weighted'))
print('FNR: ', FNR(conf_matr))
print('Profit: ', profit_score(conf_matr))

[[2156    0]
 [ 519    0]]
roc:  0.5
wf1:  0.7168028320119602
FNR:  0.19401869158878504
Profit:  0.03710575139146567
